In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

from keras.optimizers import Adam


/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                image                   id  \
1143  ../data/Train/CameraRGB/episode_0001_000087.png  episode_0001_000087   
1878                  ../data/Train/CameraRGB/355.png                  355   
1747                  ../data/Train/CameraRGB/460.png                  460   
1221  ../data/Train/CameraRGB/episode_0003_000141.png  episode_0003_000141   
2407  ../data/Train/CameraRGB/episode_0003_000130.png  episode_0003_000130   

                                                label  
1143  ../data/Train/CameraSeg/episode_0001_000087.png  
1878                  ../data/Train/CameraSeg/355.png  
1747                  ../data/Train/CameraSeg/460.png  
1221  ../data/Train/CameraSeg/episode_0003_000141.png  
2407  ../data/Train/CameraSeg/episode_0003_000130.png  


In [4]:
from models.unet import model_unet

model = model_unet(3, image_shape=(480, 480, 3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 480, 480, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 480, 480, 3)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 480, 480, 8)  224         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
from gen.generators import train_and_lab_gen_func

BATCH_SIZE = 32
model_dir = '../saved_models/unet/'

train_gen = train_and_lab_gen_func(train_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)
valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)
# test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE)

In [6]:

from train import jaccard_distance_loss
from keras.utils.training_utils import multi_gpu_model

gpus = 2

model = multi_gpu_model(model, gpus)

model.compile(optimizer = 'adam', 
                   loss = jaccard_distance_loss, 
                   metrics = ['acc', 'categorical_accuracy'])


In [ ]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/engine/training.py:2095: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Reinserting dataframe: 500 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
## Ignore next mess

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('{}/model_saved.h5'.format(model_dir))